<a href="https://www.kaggle.com/code/caiomaxximus/s-p-500-analysis-and-forecasting?scriptVersionId=162299054" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only " ../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Time series study S&P500

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


## A simple Visualization and forecast over time series data, S&P 500, (all rights reserved) the data is availble here https://fred.stlouisfed.org/series/SP500 , betteween 2018-10-01 and 2023-09-29

"The observations for the S&P 500 represent the daily index value at market close. The market typically closes at 4 PM ET, except for holidays when it sometimes closes early.

The Federal Reserve Bank of St. Louis and S&P Dow Jones Indices LLC have reached a new agreement on the use of Standard & Poors and Dow Jones Averages series in FRED. FRED and its associated services will include 10 years of daily history for Standard & Poors and Dow Jones Averages series.

The S&P 500 is regarded as a gauge of the large cap U.S. equities market. The index includes 500 leading companies in leading industries of the U.S. economy, which are publicly held on either the NYSE or NASDAQ, and covers 75% of U.S. equities. Since this is a price index and not a total return index, the S&P 500 index here does not contain dividends." 


In [ ]:
## Uncoment to download the datasets
# !wget "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1138&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=SP500&scale=left&cosd=2018-09-29&coed=2023-09-29&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily%2C%20Close&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2023-10-02&revision_date=2023-10-02&nd=2013-09-30"

#Renaming 
# !mv "/kaggle/working/fredgraph.csv?bgcolor=#e1e9f0&chart_type=line&drp=0&fo=open sans&graph_bgcolor=#ffffff&height=450&mode=fred&recession_bars=on&txtcolor=#444444&ts=12&tts=12&width=1138&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=ye" "Federal_Funds_Effective_Rate.csv"

In [ ]:
data = pd.read_csv("/kaggle/working/S&P500_DATA.csv",
                   parse_dates = ["DATE"],
                   infer_datetime_format = True
)

data = data.set_index("DATE").to_period('D')
data = pd.to_numeric(data["SP500"], errors = 'coerce')
data = data.dropna()
data = data.to_frame()


In [ ]:
data.tail()

In [ ]:
##Generating somo features to visualization

data['year'] = data.index.year
data['month'] = data.index.month
data["dayofyear"] = data.index.day_of_year
data['weekofyear'] = data.index.weekofyear
data['dayofweek'] = data.index.dayofweek
data['monthofyear'] = data.index.month
data["day"] = data.index.day
data.tail()

## Vizualing behavior during the weeks and months

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(data.index.strftime('%Y-%m-%d'), data["SP500"], linewidth=2.0)
ax.set(yticks=range(3500,5400,400),
xticks = ["2018-10-01","2023-01-29" ,"2023-09-29"])
plt.title('Values through the years')
plt.show()


In [ ]:
#Behavior of values throught the years

fig, ax = plt.subplots(figsize=(12, 8))

for year in data["year"].unique():
    subdata = data.loc[data['year'] == year]
    ax.plot(subdata['dayofyear'], subdata["SP500"], linewidth=2.0 ,label=year)
    ax
ax.set(yticks=range(2000,5400,400))
ax.legend()

plt.title('Values through the years')
plt.show()

In [ ]:

fig, ax = plt.subplots(6 ,1 , figsize=(12, 14))
for index , year in enumerate(data["year"].unique()):
    subdata = data.loc[data['year'] == year]
    
    for week in subdata["weekofyear"]:
        ## Just showing half of the weeks to avoid too much noise in the plot
        if(week % 2 == 0):
            subdataWeek = subdata.loc[subdata['weekofyear'] == week]
            ax[index].plot(subdataWeek['dayofweek'], subdataWeek["SP500"], linewidth=2.0)
    ax[index].set_xlabel(f'days of the week ({year})')
    ax[index].set(yticks=range(int(subdata["SP500"].min()),int(subdata["SP500"].max() + 400),400),
                 xticks = range(0,5))
    

fig.suptitle('Values throught the weeks', fontsize=16)
plt.tight_layout()
plt.show()
fig.savefig('subplots_weeks_per_year.png')


In [ ]:
fig, ax = plt.subplots(6 ,1 , figsize=(12, 12))
for index , year in enumerate(data["year"].unique()):
    subdata = data.loc[data['year'] == year]
    
    for month in subdata["month"]:
        subdataMonth = subdata.loc[subdata['month'] == month]
        ax[index].plot(subdataMonth['day'], subdataMonth["SP500"], linewidth=2.0)
    ax[index].set_xlabel(f'days of the month ({year})')
    ax[index].set(yticks=range(int(subdata["SP500"].min()),int(subdata["SP500"].max()) + 400,400))

fig.suptitle('Values throught the months', fontsize=16)
plt.tight_layout()
plt.show()
fig.savefig('subplots_months_per_year.png')

#### Average values in diferent years

In [ ]:
## Average values during the years
data[["year","SP500"]].groupby(['year']).agg(['mean' , 'std'])

#  Modeling some tendencies

In [ ]:
limitTrainingData = "2023-02-06"  ### The date limit to train the model
fourier1 = CalendarFourier(freq = 'M', order = 8) 
fourier2 = CalendarFourier(freq = 'Y', order = 8)
deterministic = DeterministicProcess(
    order = 1,
    seasonal = True,
    constant = True,
    index = data.loc["2018-10-01" : limitTrainingData].index,
    additional_terms = [fourier1, fourier2]
) 
dYearData = deterministic.in_sample()

In [ ]:
dYearData["lag_1"] = data.loc[dYearData.index , "SP500"].shift(1) 
dYearData.dropna(axis = 0 , inplace =True)
dYearData.head()

In [ ]:
dataToTrain =  data
model = LinearRegression(fit_intercept = False)
model.fit(dYearData ,dataToTrain.loc[dYearData.index]["SP500"])

In [ ]:
# GETTING THE MODELS'S WEIGHTS
coeficientes = model.coef_
intercept = model.intercept_
##Verifing the weights
print('Coeficientes (Weights):', coeficientes)  # Coeficiente/slopes


In [ ]:
## There are some missing dates in data, this generate the necessary amount of days 
## to match the end date of training
startDate = dYearData.iloc[-1].name.strftime('%Y-%m-%d')
endDate = "2023-10-29"
outOfSampleSize = pd.date_range(start=startDate, end= endDate).size


In [ ]:
d_out_sample = deterministic.out_of_sample(outOfSampleSize)
d_out_sample["lag_1"] = 0
dateStartPrediction = d_out_sample.iloc[0].name.strftime('%Y-%m-%d')
d_out_sample.head()

In [ ]:
##Function to generate the lag data to prediction
import warnings
warnings.filterwarnings('ignore')

def predict_out_of_sample(dateStartPrediction, endDate ,data , d_out_sample,model):
    value_predicted = 0
    lastDate = ""
    dfPredicted = pd.DataFrame(index = d_out_sample.loc[dateStartPrediction:endDate].index)

    for index , row in d_out_sample.loc[dateStartPrediction:endDate].iterrows():

        if(index.strftime('%Y-%m-%d') == dateStartPrediction ):
            d_out_sample.loc[dateStartPrediction,"lag_1"] = data.loc[dateStartPrediction, "SP500"]
            dataToPredict = d_out_sample.loc[dateStartPrediction].values.reshape(1,-1)
            value_predicted = model.predict(dataToPredict)[0]
            dfPredicted.loc[dateStartPrediction,"SP500"] = value_predicted
            lastDate = dateStartPrediction 
        else:
            d_out_sample.loc[index,"lag_1"] = dfPredicted.loc[lastDate, "SP500"]
            row_p = d_out_sample.loc[index].values.reshape(1,-1)
            value_predicted = model.predict(row_p)[0]
            dfPredicted.loc[index , "SP500"] = value_predicted
            lastDate = index
    print("Prediction complete!")
    print(dfPredicted.shape)
    return dfPredicted

dfPredicted = predict_out_of_sample(dateStartPrediction, endDate ,data , d_out_sample,model)


In [ ]:
# ## Adding a net return value to compare 
# start_date_net_ = "2023-07-05"
# end_date_net_ = "2023-08-05"
# u = dataToTrain.loc[start_date_net_: end_date_net_,"SP500"].mean()
# teta = dataToTrain.loc[start_date_net_: end_date_net_,"SP500"].std()
# err = np.random.normal(0, 1, dfPredicted.loc[end_date_net_:,"SP500"].count())

# net_return_predicited=  u + teta * err
# df_net_return = pd.DataFrame(index = dfPredicted.loc[end_date_net_:].index )
# df_net_return["SP500"] = net_return_predicited

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
common_dates = pd.date_range(start='2018-10-01', end='2023-12-31', freq='D')
ax.plot(common_dates, np.ones(common_dates.size) * 1000)
ax.plot(dataToTrain.index, dataToTrain["SP500"], linewidth=3.0,label = 'Real Value' , c= "black")
ax.plot(dYearData.index, model.predict(dYearData),linewidth = 1.0, label = 'Trained Values' )
ax.plot(dfPredicted.index, dfPredicted["SP500"], linewidth=2.0,label = 'Predicted' , c= "yellow")
# ax.plot(df_net_return.index, df_net_return["SP500"], linewidth=1.0,label = 'Net return model' , c= "red")
ax.plot()
ax.set(yticks=range(2000,5400,400))
ax.set(xticks = (["2018-01-01","2019-01-01","2020-01-01","2021-01-01","2022-01-01","2023-01-01"]))
# pd.to_datetime(dates_to_drop).to_period('D')
ax.legend()
plt.title('Values through the years')
plt.show()

# Traning second model to adjust the errors from the first model
### *Training with the predictions from 2023-02-07 to 2023-05-29; the new model than predicts from 2023-05-07 till the 11º month*

In [ ]:
index_to_compare =data.loc['2023-02-07':'2023-05-29'].index
### Measuring the error
model2Df = pd.DataFrame(index = index_to_compare)
model2Df["in"] = dfPredicted.loc[index_to_compare,"SP500"]

model2Df["SP500"] = data.loc[index_to_compare,"SP500"]
model2 = LinearRegression()
model2.fit(model2Df["in"].array.reshape(-1, 1) , model2Df["SP500"])

In [ ]:
dataStartpredHybrid = '2023-05-07'
predicitonHybrid = pd.DataFrame (index = dfPredicted.loc[dataStartpredHybrid:].index)
predicitonHybrid["SP500"] = model2.predict(dfPredicted.loc[dataStartpredHybrid:,"SP500"].array.reshape(-1, 1))


In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
common_dates = pd.date_range(start=dataStartpredHybrid, end='2023-12-31', freq='D')
ax.plot(common_dates, np.ones(common_dates.size) * 4000)
ax.plot(data.loc[dataStartpredHybrid:].index, data.loc[dataStartpredHybrid:,"SP500"], linewidth=3.0,label = 'real value' , c= "black")
# ax.plot(dYearData.index.strftime('%Y-%m-%d'), model.predict(dYearData),linewidth = 1.0, label = 'Trained Values' )
# ax.plot(df_net_return.index, df_net_return["SP500"], linewidth=1.0,label = 'Net return model' , c= "red")
ax.plot(dfPredicted.loc[dataStartpredHybrid:].index, dfPredicted.loc[dataStartpredHybrid:,"SP500"], linewidth=2.0,label = 'Predicted simple linear model' , c= "yellow")
ax.plot(predicitonHybrid.loc[dataStartpredHybrid:].index, predicitonHybrid.loc[dataStartpredHybrid:,"SP500"], linewidth=2.0,label = 'error corrected model' , c= "purple")
ax.set_xlabel('Months')
ax.plot() 
# ax.set(yticks=range(2000,5400,400))
# ax.set(xticks = (["2018-10-01", "2023-09-29" , "2023-10-29"]))
# pd.to_datetime(dates_to_drop).to_period('D')
ax.legend()
plt.title('Predictions 6 months')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
common_dates = pd.date_range(start='2018-10-01', end='2023-12-31', freq='D')
ax.plot(common_dates, np.ones(common_dates.size) * 1000)
ax.plot(data.index, data["SP500"], linewidth=3.0,label = 'Value' , c= "black")
# ax.plot(dYearData.index.strftime('%Y-%m-%d'), model.predict(dYearData),linewidth = 1.0, label = 'Trained Values' )
# ax.plot(df_net_return.index, df_net_return["SP500"], linewidth=1.0,label = 'Net return model' , c= "red")
ax.plot(dfPredicted.index, dfPredicted["SP500"], linewidth=2.0,label = 'Predicted simple linear model' , c= "yellow")
ax.plot(predicitonHybrid.index, predicitonHybrid["SP500"], linewidth=2.0,label = 'hybridModel' , c= "purple")
ax.plot() 
ax.set(yticks=range(2000,5400,400))
# ax.set(xticks = (["2018-10-01", "2023-09-29" , "2023-10-29"]))
# pd.to_datetime(dates_to_drop).to_period('D')
ax.legend()
plt.title('Predictions in comparassion with long time behavior')
plt.show()

# Visualizing errors

In [ ]:
index_dates_to_measure = data.loc[dataStartpredHybrid:].index
index_dates_to_measure.size
##
mean_error_model1 = dfPredicted.loc[index_dates_to_measure, "SP500"] - data.loc[index_dates_to_measure, "SP500"]
mean_erro_hybrid = predicitonHybrid.loc[index_dates_to_measure, "SP500"] - data.loc[index_dates_to_measure, "SP500"]

###ERROR IN PREDICITIONS
print(f"Mean Error linear model1 : {mean_error_model1.abs().mean()}")
print(f"Mean Error linear model hybrid : {mean_erro_hybrid.abs().mean()}")
print(f"Standar Deviation linear model1 : {np.std(mean_error_model1.abs())}")
print(f"Standar Deviation  model hybrid : {np.std(mean_erro_hybrid.abs())}") 

In [ ]:
##Linear model
plt.hist(mean_error_model1.abs(), bins=30, color='blue', alpha=0.7)
plt.xlabel('Absolute error')
plt.ylabel('Frequency')
plt.title('Linear model')
plt.xlim(0, 600)
plt.show()

#Linear model erro adjusted

plt.hist(mean_erro_hybrid.abs(), bins=30, color='blue', alpha=0.7)
plt.xlabel('Absolute error')
plt.ylabel('Frequency')
plt.title('Linear model error adjusted')
plt.xlim(0, 600)

plt.show()
